In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Install Spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.3/spark-3.0.3-bin-hadoop3.2.tgz

# Unzip the Spark file to the current folder
!tar xf spark-3.0.3-bin-hadoop3.2.tgz

# Install findspark
!pip install -q findspark

# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

# Import SparkSession
from pyspark.sql import SparkSession



In [ ]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

df = spark.createDataFrame([{"hello": "world"} for x in range(1000)])
df.show(5)

+-----+
|hello|
+-----+
|world|
|world|
|world|
|world|
|world|
+-----+
only showing top 5 rows



In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DateType

# Create a SparkSession
spark = SparkSession.builder \
    .appName("CreateDataFrame") \
    .getOrCreate()

# Define schema for the DataFrame
schema = StructType([
    StructField("emp_id", IntegerType(), True),
    StructField("emp_name", StringType(), True),
    StructField("job_name", StringType(), True),
    StructField("manager_id", IntegerType(), True),
    StructField("hire_date", StringType(), True),
    StructField("salary", FloatType(), True),
    StructField("commission", FloatType(), True),
    StructField("dep_id", IntegerType(), True)
])

# Create a list of tuples containing the data
data = [
    (68319, "KAYLING", "PRESIDENT", None, "1991-11-18", 6000.00, None, 1001),
    (66928, "BLAZE", "MANAGER", 68319, "1991-05-01", 2750.00, None, 3001),
    (67832, "CLARE", "MANAGER", 68319, "1991-06-09", 2550.00, None, 1001),
    (65646, "JONAS", "MANAGER", 68319, "1991-04-02", 2957.00, None, 2001),
    (67858, "SCARLET", "ANALYST", 65646, "1997-04-19", 3100.00, None, 2001),
    (69062, "FRANK", "ANALYST", 65646, "1991-12-03", 3100.00, None, 2001),
    (63679, "SANDRINE", "CLERK", 69062, "1990-12-18", 900.00, None, 2001),
    (64989, "ADELYN", "SALESMAN", 66928, "1991-02-20", 1700.00, 400.00, 3001),
    (65271, "WADE", "SALESMAN", 66928, "1991-02-22", 1350.00, 600.00, 3001),
    (66564, "MADDEN", "SALESMAN", 66928, "1991-09-28", 1350.00, 1500.00, 3001),
    (68454, "TUCKER", "SALESMAN", 66928, "1991-09-08", 1600.00, 0.00, 3001),
    (68736, "ADNRES", "CLERK", 67858, "1997-05-23", 1200.00, None, 2001),
    (69000, "JULIUS", "CLERK", 66928, "1991-12-03", 1050.00, None, 3001),
    (69324, "MARKER", "CLERK", 67832, "1992-01-23", 1400.00, None, 1001)
]

# Create DataFrame
emp_df = spark.createDataFrame(data, schema)

# Show DataFrame
emp_df.show()
department_schema = StructType([
    StructField("dep_id", IntegerType(), True),
    StructField("dep_name", StringType(), True),
    StructField("dep_location", StringType(), True)
])

# Create a list of tuples containing department data
department_data = [
    (1001, "FINANCE", "SYDNEY"),
    (2001, "AUDIT", "MELBOURNE"),
    (3001, "MARKETING", "PERTH"),
    (4001, "PRODUCTION", "BRISBANE")
]
dep_df = spark.createDataFrame(department_data, department_schema)

# Show Department DataFrame
dep_df.show()

+------+--------+---------+----------+----------+------+----------+------+
|emp_id|emp_name| job_name|manager_id| hire_date|salary|commission|dep_id|
+------+--------+---------+----------+----------+------+----------+------+
| 68319| KAYLING|PRESIDENT|      null|1991-11-18|6000.0|      null|  1001|
| 66928|   BLAZE|  MANAGER|     68319|1991-05-01|2750.0|      null|  3001|
| 67832|   CLARE|  MANAGER|     68319|1991-06-09|2550.0|      null|  1001|
| 65646|   JONAS|  MANAGER|     68319|1991-04-02|2957.0|      null|  2001|
| 67858| SCARLET|  ANALYST|     65646|1997-04-19|3100.0|      null|  2001|
| 69062|   FRANK|  ANALYST|     65646|1991-12-03|3100.0|      null|  2001|
| 63679|SANDRINE|    CLERK|     69062|1990-12-18| 900.0|      null|  2001|
| 64989|  ADELYN| SALESMAN|     66928|1991-02-20|1700.0|     400.0|  3001|
| 65271|    WADE| SALESMAN|     66928|1991-02-22|1350.0|     600.0|  3001|
| 66564|  MADDEN| SALESMAN|     66928|1991-09-28|1350.0|    1500.0|  3001|
| 68454|  TUCKER| SALESMA

In [ ]:
emp_df.createOrReplaceTempView("employees")
dep_df.createOrReplaceTempView("departments")

result = spark.sql("SELECT emp_name, dep_name FROM employees JOIN departments ON employees.dep_id = departments.dep_id")
result.show()


+--------+---------+
|emp_name| dep_name|
+--------+---------+
|   BLAZE|MARKETING|
|  ADELYN|MARKETING|
|    WADE|MARKETING|
|  MADDEN|MARKETING|
|  TUCKER|MARKETING|
|  JULIUS|MARKETING|
| KAYLING|  FINANCE|
|   CLARE|  FINANCE|
|  MARKER|  FINANCE|
|   JONAS|    AUDIT|
| SCARLET|    AUDIT|
|   FRANK|    AUDIT|
|SANDRINE|    AUDIT|
|  ADNRES|    AUDIT|
+--------+---------+



In [ ]:
result = spark.sql("SELECT employees.emp_name, managers.emp_name \
                    FROM employees \
                    JOIN employees managers ON employees.manager_id = managers.emp_id \
                    WHERE employees.manager_id IS NOT NULL")
result.show()

+--------+--------+
|emp_name|emp_name|
+--------+--------+
|  MARKER|   CLARE|
|  ADELYN|   BLAZE|
|    WADE|   BLAZE|
|  MADDEN|   BLAZE|
|  TUCKER|   BLAZE|
|  JULIUS|   BLAZE|
|SANDRINE|   FRANK|
| SCARLET|   JONAS|
|   FRANK|   JONAS|
|   BLAZE| KAYLING|
|   CLARE| KAYLING|
|   JONAS| KAYLING|
|  ADNRES| SCARLET|
+--------+--------+



In [ ]:
result = spark.sql("SELECT employees.emp_name, COALESCE(managers.emp_name, 'No Manager') AS manager_name \
                    FROM employees \
                    LEFT JOIN employees managers ON employees.manager_id = managers.emp_id")

result.show()

+--------+------------+
|emp_name|manager_name|
+--------+------------+
| KAYLING|  No Manager|
|  MARKER|       CLARE|
|  ADELYN|       BLAZE|
|    WADE|       BLAZE|
|  MADDEN|       BLAZE|
|  TUCKER|       BLAZE|
|  JULIUS|       BLAZE|
|SANDRINE|       FRANK|
| SCARLET|       JONAS|
|   FRANK|       JONAS|
|   BLAZE|     KAYLING|
|   CLARE|     KAYLING|
|   JONAS|     KAYLING|
|  ADNRES|     SCARLET|
+--------+------------+



In [ ]:
result = spark.sql("SELECT employees.emp_name \
                    FROM employees \
                    LEFT JOIN employees managers ON employees.manager_id = managers.emp_id \
                    WHERE managers.emp_id IS NULL")

result.show()

+--------+
|emp_name|
+--------+
| KAYLING|
+--------+



In [ ]:
result = spark.sql("SELECT managers.emp_name, COUNT(employees.emp_id) AS num_employees \
                    FROM employees managers \
                    LEFT JOIN employees ON managers.emp_id = employees.manager_id \
                    GROUP BY managers.emp_id, managers.emp_name \
                    ORDER BY num_employees DESC \
                    LIMIT 1")
result.show()

+--------+-------------+
|emp_name|num_employees|
+--------+-------------+
|   BLAZE|            5|
+--------+-------------+

